In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, \
                                        BatchNormalization, Embedding, Masking, Bidirectional, Conv1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from sklearn.utils import class_weight, shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.inspection import permutation_importance

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from xgboost import XGBRegressor, XGBClassifier

import scipy.stats as stats
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
BATCH_SIZE = 32
EPOCHS = 300

from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import cx_Oracle
import pandas as pd
from sqlalchemy import create_engine
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib
import shap

In [2]:
def generate_time(start_date:str, end_date:str, hour:int):
        start = datetime.strptime(start_date, '%d/%m/%Y')
        end = datetime.strptime(end_date, '%d/%m/%Y')

        dates = []
        while start<=end:
            row = [start]
            dates.append(row)
            start += timedelta(hours=hour)

        return pd.DataFrame(dates, columns=['TIMESTAMP'])
    
def query_status():
    try:
        oracle_string = "oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}"
        engine = create_engine(
            oracle_string.format(
                username = 'TFM4CEBERUS',
                password = 'TFM4CEBERUS',
                hostname = 'ome-db.bth.infineon.com',
                port = '1538',
                database = 'ome'
                )
            )
    except Exception as e:
        print(str(e))

    query = f"""select EQ_ID, TIMESTAMP_START, TIMESTAMP_END, DURATION, STATE_NAME, LEVEL3_NAME, LEVEL3 
            from (SELECT
              eq.eq_id, eq.name, eq.eq_type_ident
            , data.timestamp_start,data.timestamp_end
            , ROUND((data.timestamp_end - data.timestamp_start)*24*60*60,0) AS Duration
            , data.tr25_3_status,data.tr25_4_status,data.tr25_5_status,data.eq_status
            , level5s.state_name
            , level5.state_name Level5_Name, level5.state_sign Level5
            , level4.state_name Level4_Name, level4.state_sign Level4
            , level3.state_name Level3_Name, level3.state_sign Level3
            ,mh.device
            ,mh.package,
            mh.lotid as lot,
            mh.product,
            mh.operation

            FROM OMEDATA.EQUIPMENT_STATE_HISTORY data
            , OMEADMIN.EQUIPMENT_INSTANCES eq
            , V_EQ_STATES level5s
            , OMEADMIN.DEF_STANDARD_STATEMODEL level5
            , OMEADMIN.DEF_STANDARD_STATEMODEL level4
            , OMEADMIN.DEF_STANDARD_STATEMODEL level3
            , OMEDATA.METAKEY_HISTORY mh

            WHERE data.eq_ident  = eq.eq_ident
            AND  data.eq_status = level5s.state_ident(+)
            AND level5.state_ident = data.tr25_5_status
            AND level4.state_ident = data.tr25_4_status
            AND level3.state_ident = data.tr25_3_status
            AND  data.metakey_ident =mh.ident(+)
            and data.timestamp_start > sysdate - 1050)
            where eq_id = 'WBA127'
            ORDER BY TIMESTAMP_START"""

    status = pd.read_sql(query, engine)
    status.columns = map(lambda x: str(x).upper(), status.columns) 

    return status

def aggregate(timeframe_table, lookback_window, status_table):
    alarm_df = pd.DataFrame()
    statename_df = pd.DataFrame(columns=status_table["STATE_NAME"].unique())

    for idx, row in timeframe_table.iterrows():
        end = row["TIMESTAMP"]
        start = end - timedelta(hours=lookback_window)

        ## count the frequencies of each alarm
#             filtered_alarm = alarm_table.loc[(alarm_table["DT_SET"] >= start) & (alarm_table["DT_SET"] <= end)]
#             alarm_freq_table = filtered_alarm["Alarm ID"].value_counts().to_frame().T.reset_index(drop=True)
#             alarm_df = pd.concat([alarm_df, alarm_freq_table], axis=0)

        ## count the frequencies of each statename, include everything since feature engineering would be performed
        filtered_statename = status_table.loc[(status_table["TIMESTAMP_START"] >= start) & (status_table["TIMESTAMP_START"] <= end)]
        unique = filtered_statename["STATE_NAME"].unique()
        status_dict = {key:int(sum(filtered_statename.loc[filtered_statename.STATE_NAME==key]["DURATION"])) 
                       for key in unique}
        
        statename_df = statename_df.append(status_dict, ignore_index=True)
            
    statename_df = statename_df.fillna(0)
    return statename_df

    ## convert all columns from float to int
    cols = statename_df.columns
    statename_df[cols] = statename_df[cols].astype(int)
#     statename_df["EQUIPMENT"] = "WBA124"
    return statename_df


def major_down(input_df, status_table, hour, threshold): 
        hour = pd.Timedelta(hours=hour)
        major_down = []

        for idx, row in input_df.iterrows():
            start = row['TIMESTAMP']
            end = start+hour
            frame = status_table[(status_table['TIMESTAMP_START']>start) & (status_table['TIMESTAMP_START']<end)]
            UD = frame.loc[frame['LEVEL3']=='UDT']

            if len(UD) == 0: #no record within this 6 hours:
                major_down.append(0)
            else:
                time_diff = (UD['TIMESTAMP_END']-UD['TIMESTAMP_START']).dt.seconds
                if any(time_diff>=threshold): #threshold = 3600s
                    major_down.append(1)
                else:
                    major_down.append(0)
        return np.array(major_down)

In [3]:
wba127_status_table = query_status()

In [168]:
answer_col = wba127_status_table.loc[(wba127_status_table.LEVEL3 == "UDT") & 
                       (wba127_status_table.DURATION >= 3600)]["STATE_NAME"].unique()
answer_col

array(['Non LIFTED STICK BOND', 'Waiting For Technician', 'SHORT TAIL',
       'WIRE BREAK', 'POOR RECOGNITION', 'Waiting For Repair',
       'NO BONDING', 'NSOP/NSOL', 'GOLF BALL BOND', 'Waiting For Spares',
       'Fail Surveillance', 'Utility Problem', 'BALL PLACEMENT',
       'BAD WEDGE FORM', 'INDEXER PROBLEM', 'Waiting For Operator',
       'HANG UP', 'Waiting for Setup', 'PC Buyoff Failed', 'IT Problem',
       'BROKEN WIRE'], dtype=object)

In [143]:
wba127_timeframe_table = generate_time('5/12/2018', '31/8/2021', 24)

In [144]:
status_df = aggregate(wba127_timeframe_table, 12, wba127_status_table)
status_df

,Normal Production,SHORT TAIL,Waiting For Response,Change Capillary,Change Wire,NO BONDING,Waiting For Operator,No Material,Visual Inspection,Waiting For Technician,...,Maintenance,Waiting for Setup,LOADER/UNLOADER PROBLEM,APC Input Violation,Engineering Evaluation,Undefined Waiting,MC Banned,FullStock,Engineering EE,Material Problem
0,35441.0,22.0,5767.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33149.0,119.0,4389.0,0.0,0.0,0.0,98.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36245.0,67.0,1628.0,5149.0,622.0,0.0,153.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,38142.0,37.0,5126.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32574.0,971.0,2736.0,5859.0,239.0,9.0,67.0,0.0,101.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,23572.0,17402.0,1302.0,0.0,0.0,42.0,281.0,0.0,164.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,24720.0,484.0,1565.0,0.0,1846.0,0.0,6122.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,35891.0,2934.0,2369.0,0.0,0.0,0.0,191.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,34552.0,0.0,1357.0,3134.0,1620.0,0.0,110.0,0.0,107.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [181]:
status_df["target"] = target
status_df.corr()

,Normal Production,SHORT TAIL,Waiting For Response,Change Capillary,Change Wire,NO BONDING,Waiting For Operator,No Material,Visual Inspection,Waiting For Technician,...,Waiting for Setup,LOADER/UNLOADER PROBLEM,APC Input Violation,Engineering Evaluation,Undefined Waiting,MC Banned,FullStock,Engineering EE,Material Problem,target
Normal Production,1.000000,-0.008824,0.522516,0.224880,0.240186,0.038340,0.128077,-0.058737,0.075999,-0.094099,...,-0.116727,0.050234,-0.008404,0.021119,0.002338,0.018829,0.013815,-0.026702,0.028843,-0.003102
SHORT TAIL,-0.008824,1.000000,0.036101,-0.046744,-0.019630,0.076619,0.029987,0.009914,-0.003968,0.022074,...,-0.043350,-0.008623,-0.015004,-0.013924,0.014108,-0.012664,-0.015067,-0.015067,-0.014651,0.118078
Waiting For Response,0.522516,0.036101,1.000000,0.089864,0.059970,0.050667,0.098490,-0.043751,-0.012778,-0.032100,...,-0.081548,-0.018106,0.016985,-0.014157,0.003472,-0.023256,0.007296,0.049395,-0.024503,0.004857
Change Capillary,0.224880,-0.046744,0.089864,1.000000,0.100784,-0.034293,-0.009759,-0.024823,-0.000703,-0.025064,...,-0.045620,0.039614,-0.012551,-0.004583,-0.015197,-0.012529,-0.012529,-0.012529,-0.012529,-0.035307
Change Wire,0.240186,-0.019630,0.059970,0.100784,1.000000,0.002631,0.022892,-0.015453,0.057748,-0.030261,...,-0.022694,-0.013244,-0.013241,0.084433,0.009964,0.257599,-0.002391,-0.013244,0.010042,-0.052700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MC Banned,0.018829,-0.012664,-0.023256,-0.012529,0.257599,-0.004787,0.015249,-0.002303,0.032315,-0.004329,...,-0.003724,-0.001000,-0.001002,-0.001000,-0.001213,1.000000,-0.001000,-0.001000,-0.001000,-0.009879
FullStock,0.013815,-0.015067,0.007296,-0.012529,-0.002391,-0.004787,0.020153,-0.002303,-0.008844,-0.004329,...,-0.003724,-0.001000,-0.001002,-0.001000,-0.001213,-0.001000,1.000000,-0.001000,-0.001000,-0.009879
Engineering EE,-0.026702,-0.015067,0.049395,-0.012529,-0.013244,-0.004787,-0.012554,0.010407,0.023086,-0.004329,...,0.092284,-0.001000,-0.001002,-0.001000,-0.001213,-0.001000,-0.001000,1.000000,-0.001000,-0.009879
Material Problem,0.028843,-0.014651,-0.024503,-0.012529,0.010042,-0.003569,-0.003381,-0.002303,-0.008844,-0.004329,...,-0.003724,-0.001000,-0.001002,-0.001000,-0.001213,-0.001000,-0.001000,-0.001000,1.000000,-0.009879


In [157]:
col_robust = []
for col in status_df:
    if  len(status_df[col].unique()) > 10:
        print(col, len(status_df[col].unique()))
        col_robust.append(col)

Normal Production 823
SHORT TAIL 536
Waiting For Response 747
Change Capillary 250
Change Wire 283
NO BONDING 126
Waiting For Operator 474
No Material 80
Visual Inspection 279
Waiting For Technician 69
Non LIFTED STICK BOND 516
WIRE BREAK 102
Change Lot 67
Surveillance 198
Change Device 98
PC Conform 32
POOR RECOGNITION 33
PlanIdle 71
NoWIP 68
INDEXER PROBLEM 124
Fail Surveillance 94
GOLF BALL BOND 11
Machine Failure 15
Engineering PRE 18
HANG UP 27
No Operator 18
NSOP/NSOL 367
Waiting For Repair 183
Change Magazine 18
PC Buyoff 162
PC Buyoff Passed 98
PC Buyoff Failed 18
BAD WEDGE FORM 11
Meeting 19
Temperature Checking 24
Waiting for Setup 42


In [146]:
target = major_down(wba127_timeframe_table, wba127_status_table, 6, 3600)

In [147]:
len(status_df) == len(target)

True

In [169]:
# train = status_df[col_robust].values
# train_pct = int(0.8*len(train))

# X_train, X_val = train[:train_pct], train[train_pct:]
# y_train, y_val = target[:train_pct], target[train_pct:]

# remove columns which are suppsoed to tell major down
train = status_df.drop(answer_col, axis=1)

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, stratify=target)
print(Counter(y_train), Counter(y_val))

Counter({0: 729, 1: 71}) Counter({0: 183, 1: 18})


In [178]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Normal Production        1001 non-null   float64
 1   Waiting For Response     1001 non-null   float64
 2   Change Capillary         1001 non-null   float64
 3   Change Wire              1001 non-null   float64
 4   No Material              1001 non-null   float64
 5   Visual Inspection        1001 non-null   float64
 6   Change Lot               1001 non-null   float64
 7   Surveillance             1001 non-null   float64
 8   NO BALL SIZE             1001 non-null   float64
 9   Change Device            1001 non-null   float64
 10  PC Conform               1001 non-null   float64
 11  PlanIdle                 1001 non-null   float64
 12  NoWIP                    1001 non-null   float64
 13  Machine Failure          1001 non-null   float64
 14  Non Schedule Time       

In [171]:
oversample = SMOTE()
smote_X, smote_y = oversample.fit_resample(X_train, y_train)
print(Counter(smote_y))

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)
# weights ={}
# for idx, val in enumerate(class_weights):
#     weights[idx] = val
# weights

Counter({0: 729, 1: 729})


In [177]:
dt_model = DecisionTreeClassifier(random_state=42) # decision tree has no n_jobs parameter
xgb_model = XGBClassifier(random_state=42, n_jobs=-1)
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

def to_labels(y_scores, threshold):
    return (y_scores >= threshold).astype('int')

models = [xgb_model]
for model in models:
    start = datetime.now()
    model.fit(smote_X, smote_y)
    pred = model.predict(X_val)
    y_score = model.predict_proba(X_val)[:,1]

#     end = datetime.now()
#     print(f"Training took {(end-start).seconds} seconds")
#     print(confusion_matrix(y_val, pred))
#     print(f"Prediction Accuracy for {type(model).__name__} is {accuracy_score(y_val, pred)}")
print(confusion_matrix(y_val, pred), accuracy_score(y_val, pred))
# print("Model out of bag score = ", rf_model.oob_score_)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

print("ROC AUC score = ", roc_auc_score(y_val, y_score))
fpr, tpr, thresholds = roc_curve(y_val, y_score)
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))

testest = to_labels(y_score, best_thresh) # best thresh optimizes recall
confusion_matrix(y_val, testest), accuracy_score(y_val, testest)

[16:14:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[167  16]
 [ 16   2]] 0.8407960199004975
ROC AUC score =  0.5537340619307833
Best Threshold=0.009583


(array([[ 52, 131],
        [  1,  17]]),
 0.34328358208955223)

In [175]:
# WBA124 after tuning threshold 0.74
# (array([[138,  45],
#        [  7,  11]]),

# WBA127 after tuning threshold 0.59
# (array([[110,  73],
#         [  9,   9]]),